In [14]:
import nltk
import random
import numpy as np

from bs4 import BeautifulSoup

In [5]:
# load the reviews
# data courtesy of http://www.cs.jhu.edu/~mdredze/datasets/sentiment/index2.html
positive_reviews = BeautifulSoup(open('../large_files/electronics/positive.review', encoding='utf-8').read())
positive_reviews = positive_reviews.findAll('review_text')

In [87]:
# extract trigrams and insert into dictionary
# (w1, w3) is the key, [ w2 ] are the values
trigrams = {}
for review in positive_reviews:
    s = review.text.lower()
    tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(tokens) - 2):
        k = (tokens[i], tokens[i+2])
        if k not in trigrams:
            trigrams[k] = []
        trigrams[k].append(tokens[i+1])

# turn each array of middle-words into a probability vector
trigram_probabilities = {}
for k, words in trigrams.items():
    # create a dictionary of word -> count
    if len(set(words)) > 1:
        # only do this when there are different possibilities for a middle word
        d = {}
        n = 0
        for w in words:
            if w not in d:
                d[w] = 0
            d[w] += 1
            n += 1
        for w, c in d.items():
            d[w] = float(c) / n

        trigram_probabilities[k] = d

In [89]:
# View some probability of model
i = 0
for k, v in trigram_probabilities.items():
    print(k, v)
    
    i +=1
    if i >= 5:
        break

('i', 'this') {'purchased': 0.12422360248447205, 'bought': 0.3105590062111801, 'recomend': 0.006211180124223602, 'made': 0.012422360248447204, 'picked': 0.018633540372670808, 'say': 0.006211180124223602, 'use': 0.055900621118012424, 'had': 0.006211180124223602, 'got': 0.055900621118012424, 'think': 0.037267080745341616, 'ordered': 0.012422360248447204, 'matched': 0.006211180124223602, 'noticed': 0.006211180124223602, 'thought': 0.006211180124223602, 'recommend': 0.024844720496894408, 'choose': 0.006211180124223602, 'like': 0.012422360248447204, 'found': 0.055900621118012424, 'did': 0.018633540372670808, 'find': 0.024844720496894408, 'set': 0.006211180124223602, 'love': 0.024844720496894408, 'hold': 0.006211180124223602, 'received': 0.012422360248447204, 'have': 0.018633540372670808, 'buy': 0.006211180124223602, 'chose': 0.012422360248447204, 'put': 0.006211180124223602, 'used': 0.012422360248447204, 'give': 0.006211180124223602, 'believe': 0.006211180124223602, 'mention': 0.00621118012

In [104]:
# def random_sample(d):
#     # choose a random sample from dictionary where values are the probabilities
#     r = random.random()
#     cumulative = 0
#     for w, p in d.items():
#         cumulative += p
#         if r < cumulative:
#             return w

def random_sample(d):
    all_words = [k for k in d.keys()]
    probs = [v for v in d.values()]
    
    w = np.random.choice(all_words, 1, p=probs)
    return w[0]

def run_spinner():
    review = random.choice(positive_reviews)
    s = review.text.lower()
    original_tokens = nltk.tokenize.word_tokenize(s)
    new_tokens = nltk.tokenize.word_tokenize(s)
    for i in range(len(original_tokens) - 2):
        if random.random() < 0.2: # 20% chance of replacement
            k = (original_tokens[i], original_tokens[i+2])
            if k in trigram_probabilities:
                w = random_sample(trigram_probabilities[k])
                new_tokens[i+1] = "-{}-".format(w)
                original_tokens[i+1] = "-{}-".format(original_tokens[i+1])
    print("Orignal:")
    print(" ".join(original_tokens))
    
    print("Spun:")
    print(" ".join(new_tokens))

In [107]:
for i in range(5):
    run_spinner()
    print('\n')

Orignal:
i bought this -speaker- cable -because- i wanted it to fall in a straight line and -hug- the walls without curling all over the -place- . this cable does just that . i 'm very happy -with- it . if that -is- your goal too , i can recommend this cable . regarding the long standing debate of whether monster cable sounds better than cheaper cable , i 'm not an audio engineer and do n't have an opinion
Spun:
i bought this -thx- cable -.- i wanted it to fall in a straight line and -asked- the walls without curling all over the -interference- . this cable does just that . i 'm very happy -with- it . if that -bought- your goal too , i can recommend this cable . regarding the long standing debate of whether monster cable sounds better than cheaper cable , i 'm not an audio engineer and do n't have an opinion


Orignal:
i bought this reader because i -wanted- to avoid having my camera and all its cable -attached- to my pc to download the pictures ; and -with- this reader i could leave m